In [1]:
intent = ['看不懂的','闲聊','商品/品类','搜优惠','搜活动/会场','其他']
slot = ['日期','商品名','品牌','店铺','颜色','价格','数量','属性']
slot_dict = {'日期':'DATE','商品/品类':'PRODUCT','品牌':'BRAND','店铺':'SHOP','颜色':'COLOR','价格':'PRICE','数量':'AMOUT','属性':'ATTRIBUTE'}

In [2]:
tmp_list = []
tmp_list.append("<PAD>")
tmp_list.append("O")
for item in slot_dict.values():
    tmp_list.append("B" + "-" + item)
    tmp_list.append("I" + "-" + item)
print(tmp_list)

['<PAD>', 'O', 'B-DATE', 'I-DATE', 'B-PRODUCT', 'I-PRODUCT', 'B-BRAND', 'I-BRAND', 'B-SHOP', 'I-SHOP', 'B-COLOR', 'I-COLOR', 'B-PRICE', 'I-PRICE', 'B-AMOUT', 'I-AMOUT', 'B-ATTRIBUTE', 'I-ATTRIBUTE']


In [3]:
import pandas 
import os
def read_data_by_folder(folder):
    text_list = []
    slot_list = []
    
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            text_list.append(tmp[0])

            tmp_slot_list = []
            for idx, value in enumerate(tmp):
                if idx>2 and idx %2 == 1 and idx < len(tmp)-2:
                    slot_type = tmp[idx]
                    slot_value = tmp[idx+1]
                    if len(str(slot_type).strip()) > 1:
                        try:
                            tmp_slot_list.append((slot_dict[slot_type], slot_value))
                        except:
                            tmp_slot_list.append(('ATTRIBUTE', slot_value))
            slot_list.append(tmp_slot_list)
    return text_list, slot_list

In [4]:
text_list, slot_list = \
read_data_by_folder("/Users/sunhongchao/Desktop/1111/all_data_till_0916/")

In [5]:
output = open('label.txt', encoding='utf-8', mode='w')

for text, tmp_slot_list in zip(text_list, slot_list):
    text_label = ['O']*len(text)
    for slot in tmp_slot_list:
        idx = text.index(slot[1])
        if idx < 0:
            continue
        slot_type = slot[0]
        text_label[idx] = "B" + "-" + slot_type
        
        for idx in range(idx+1, idx+len(slot[1]), 1):
            text_label[idx] = "I" + "-" + slot_type
    
    for char, label in zip(text, text_label):
        output.write(char + '\t' + label + '\n')
    output.write('\n')
    
output.close()

In [6]:
import pandas 
import os
def read_clf_data_by_folder(folder):
    clf_list = []
    
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            clf_list.append(tmp[1]+ '\t' +tmp[0])
       
    return clf_list

In [7]:
import pandas 
from sklearn.model_selection import train_test_split
import random

clf_list = \
read_clf_data_by_folder("/Users/sunhongchao/Desktop/1111/all_data_till_0916/")

random.shuffle(clf_list)

cut_idx = int(len(clf_list) * 0.9)

train_clf_list = clf_list[:cut_idx]
test_clf_list = clf_list[cut_idx:]

clf_output = open('train_clf.txt', encoding='utf-8', mode='w')
for item in train_clf_list:
    clf_output.write(item + '\n')

clf_output = open('test_clf.txt', encoding='utf-8', mode='w')
for item in test_clf_list:
    clf_output.write(item + '\n')